# Data Analysis of IPL[2008-2020]
In this project, we are going to analyse several different insights about IPL matches played between 2008-2020 using PySpark and SparkSQL.

Lets import PySpark and load the datasets

In [1]:
import spark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("") \
    .getOrCreate()


df = spark.read.csv('ipl_ball_by_ball.csv',header=True)
df1 = spark.read.csv('ipl_matches.csv',header=True)
df2 = spark.read.csv('ipl_venue.csv',header=True)

Lets also create SQL tables using dataframes

In [2]:
df.createOrReplaceTempView("balls")
spark.sql(
"""
select * from balls
"""
)

df1.createOrReplaceTempView("matches")
spark.sql(
"""
select * from matches
"""
)

df2.createOrReplaceTempView("venue")
spark.sql(
"""
select * from venue
"""
)

DataFrame[venue_id: string, venue: string, city: string]

# Lets look at dataset first

In [3]:
df.show(vertical=True)

-RECORD 0--------------------------------
 match_id         | 419157               
 inning           | 2                    
 overs            | 16                   
 ball             | 4                    
 batsman          | V Kohli              
 non_striker      | R Vinay Kumar        
 bowler           | CRD Fernando         
 batsman_runs     | 0                    
 extra_runs       | 0                    
 total_runs       | 0                    
 non_boundary     | 0                    
 is_wicket        | 0                    
 dismissal_kind   | NA                   
 player_dismissed | NA                   
 fielder          | NA                   
 extras_type      | NA                   
 batting_team     | Royal Challengers... 
 bowling_team     | Mumbai Indians       
-RECORD 1--------------------------------
 match_id         | 419157               
 inning           | 2                    
 overs            | 16                   
 ball             | 5             

In [4]:
df1.show(vertical=True)

-RECORD 0-------------------------------
 match_id        | 335982               
 date            | 2008-04-18           
 player_of_match | BB McCullum          
 venue_id        | 35                   
 neutral_venue   | 0                    
 team1           | Royal Challengers... 
 team2           | Kolkata Knight Ri... 
 toss_winner     | Royal Challengers... 
 toss_decision   | field                
 winner          | Kolkata Knight Ri... 
 result          | runs                 
 result_margin   | 140                  
 eliminator      | N                    
 method          | NA                   
 umpire1         | Asad Rauf            
 umpire2         | RE Koertzen          
-RECORD 1-------------------------------
 match_id        | 335983               
 date            | 2008-04-19           
 player_of_match | MEK Hussey           
 venue_id        | 31                   
 neutral_venue   | 0                    
 team1           | Kings XI Punjab      
 team2          

In [5]:
df2.show()

+--------+--------------------+--------------+
|venue_id|               venue|          city|
+--------+--------------------+--------------+
|       1|Dr. Y.S. Rajasekh...| Visakhapatnam|
|       2|Sharjah Cricket S...|       Sharjah|
|       3|JSCA Internationa...|        Ranchi|
|       4|Saurashtra Cricke...|        Rajkot|
|       5|Shaheed Veer Nara...|        Raipur|
|       6|Maharashtra Crick...|          Pune|
|       7|Subrata Roy Sahar...|          Pune|
|       8|    St George's Park|Port Elizabeth|
|       9|Vidarbha Cricket ...|        Nagpur|
|      12|   Brabourne Stadium|        Mumbai|
|      13|Dr DY Patil Sport...|        Mumbai|
|      14|    Wankhede Stadium|        Mumbai|
|      15|        Eden Gardens|       Kolkata|
|      16|       Nehru Stadium|         Kochi|
|      17|De Beers Diamond ...|     Kimberley|
|      18|          Green Park|        Kanpur|
|      19|New Wanderers Sta...|  Johannesburg|
|      20|Sawai Mansingh St...|        Jaipur|
|      21|Hol

# Data Preparation and Cleaning

Lets convert some columns to integer type  

In [6]:
from pyspark.sql.types import IntegerType
df = df.withColumn("is_wicket", df["is_wicket"].cast(IntegerType()))
df = df.withColumn("batsman_runs", df["batsman_runs"].cast(IntegerType()))
df = df.withColumn("extra_runs", df["extra_runs"].cast(IntegerType()))
df = df.withColumn("total_runs", df["total_runs"].cast(IntegerType()))


Lets look out summary of our the data

In [7]:
df.summary().show(vertical=True)

-RECORD 0--------------------------------
 summary          | count                
 match_id         | 193468               
 inning           | 193468               
 overs            | 193468               
 ball             | 193468               
 batsman          | 193468               
 non_striker      | 193468               
 bowler           | 193468               
 batsman_runs     | 193468               
 extra_runs       | 193468               
 total_runs       | 193468               
 non_boundary     | 193468               
 is_wicket        | 193468               
 dismissal_kind   | 193468               
 player_dismissed | 193468               
 fielder          | 193468               
 extras_type      | 193468               
 batting_team     | 193468               
 bowling_team     | 193468               
-RECORD 1--------------------------------
 summary          | mean                 
 match_id         | 756768.8084386048    
 inning           | 1.482131411913

# Lets check out if there are any nan values or missing data?

In [8]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0---------------
 match_id         | 0   
 inning           | 0   
 overs            | 0   
 ball             | 0   
 batsman          | 0   
 non_striker      | 0   
 bowler           | 0   
 batsman_runs     | 0   
 extra_runs       | 0   
 total_runs       | 0   
 non_boundary     | 0   
 is_wicket        | 0   
 dismissal_kind   | 0   
 player_dismissed | 0   
 fielder          | 0   
 extras_type      | 0   
 batting_team     | 0   
 bowling_team     | 0   



In [9]:
from pyspark.sql.functions import isnan, when, count, col
df1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df1.columns]).show(vertical=True)

-RECORD 0--------------
 match_id        | 0   
 date            | 0   
 player_of_match | 0   
 venue_id        | 0   
 neutral_venue   | 0   
 team1           | 0   
 team2           | 0   
 toss_winner     | 0   
 toss_decision   | 0   
 winner          | 0   
 result          | 0   
 result_margin   | 0   
 eliminator      | 0   
 method          | 0   
 umpire1         | 0   
 umpire2         | 0   



There aren't any  missing or nan values 

# Lets start Analysis

# Which players have most sixes in IPL?

In [10]:
six=df.filter(df.batsman_runs=='6')
six.groupBy('batsman').count().sort('count',ascending=False).show(10)

+--------------+-----+
|       batsman|count|
+--------------+-----+
|      CH Gayle|  349|
|AB de Villiers|  235|
|      MS Dhoni|  216|
|     RG Sharma|  214|
|       V Kohli|  202|
|    KA Pollard|  198|
|     DA Warner|  195|
|      SK Raina|  194|
|     SR Watson|  190|
|    RV Uthappa|  163|
+--------------+-----+
only showing top 10 rows



# Which players have most fours in IPL?

In [11]:
four=df.filter(df.batsman_runs=='4')
four.groupBy('batsman').count().sort('count',ascending=False).show(10)

+--------------+-----+
|       batsman|count|
+--------------+-----+
|      S Dhawan|  591|
|     DA Warner|  510|
|       V Kohli|  504|
|      SK Raina|  493|
|     G Gambhir|  492|
|     RG Sharma|  458|
|    RV Uthappa|  454|
|     AM Rahane|  416|
|AB de Villiers|  390|
|      CH Gayle|  384|
+--------------+-----+
only showing top 10 rows



# Which players have most runs in boundaries?

In [12]:
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import count as _count

four_six=df.filter(df.batsman_runs.isin(4,6))
four_and_six=four_six.groupBy('Batsman').agg(_sum('batsman_runs').alias("Runs in boundaries")).sort('Runs in boundaries',ascending=False)
four_and_six.show()

+--------------+------------------+
|       Batsman|Runs in boundaries|
+--------------+------------------+
|      CH Gayle|              3630|
|       V Kohli|              3228|
|     DA Warner|              3210|
|      SK Raina|              3136|
|     RG Sharma|              3116|
|      S Dhawan|              3018|
|AB de Villiers|              2970|
|    RV Uthappa|              2794|
|     SR Watson|              2644|
|      MS Dhoni|              2548|
|     G Gambhir|              2322|
|    KD Karthik|              2138|
|     AM Rahane|              2120|
|     AT Rayudu|              2024|
|     YK Pathan|              2002|
|    KA Pollard|              1972|
|      V Sehwag|              1972|
|   BB McCullum|              1952|
|  Yuvraj Singh|              1762|
|      PA Patel|              1754|
+--------------+------------------+
only showing top 20 rows



# Who are the highest run getters in ipl?

In [13]:
total_runs=df.groupBy('Batsman').agg(_sum('batsman_runs').alias("Runs")).sort("Runs",ascending=False)
total_runs.show()


+--------------+----+
|       Batsman|Runs|
+--------------+----+
|       V Kohli|5878|
|      SK Raina|5368|
|     DA Warner|5254|
|     RG Sharma|5230|
|      S Dhawan|5197|
|AB de Villiers|4849|
|      CH Gayle|4772|
|      MS Dhoni|4632|
|    RV Uthappa|4607|
|     G Gambhir|4217|
|     AM Rahane|3933|
|     SR Watson|3874|
|    KD Karthik|3823|
|     AT Rayudu|3659|
|     MK Pandey|3268|
|     YK Pathan|3204|
|    KA Pollard|3023|
|   BB McCullum|2880|
|      PA Patel|2848|
|  Yuvraj Singh|2750|
+--------------+----+
only showing top 20 rows



# Which all players have highest number of runs scored by boundaries?

In [14]:
boundary=four_and_six.join(total_runs,on='Batsman')
boundary_runs=boundary.withColumn("Percentage of runs by boundary", col('Runs in boundaries')*100/col('Runs')).sort('Percentage of runs by boundary',ascending=False)
boundary_runs.select(col('Batsman'),col('Percentage of runs by boundary')).show(10)

+-----------+------------------------------+
|    Batsman|Percentage of runs by boundary|
+-----------+------------------------------+
|   BA Bhatt|                         100.0|
| GD McGrath|                         100.0|
|   P Chopra|                         100.0|
|   RS Sodhi|                         100.0|
| Avesh Khan|                         100.0|
|  VRV Singh|                         100.0|
|   L Ronchi|             88.23529411764706|
|   Umar Gul|             87.17948717948718|
|J Arunkumar|             86.95652173913044|
|MDKJ Perera|             85.71428571428571|
+-----------+------------------------------+
only showing top 10 rows



This won't really doesn't gives us a clear picture, Thats why understanding the data is very important. So lets only find players who have scored more than 1000 runs

In [15]:
result_10=boundary_runs.filter(boundary_runs.Runs > 1000)
result_10.select(col('Batsman'),col('Percentage of runs by boundary')).show()

+------------+------------------------------+
|     Batsman|Percentage of runs by boundary|
+------------+------------------------------+
|  AD Russell|             78.70797626895188|
|    CH Gayle|             76.06873428331936|
|AC Gilchrist|             72.88545190913484|
|    V Sehwag|             72.28739002932551|
|    DR Smith|             70.52410901467505|
|     CA Lynn|                      69.53125|
|   SR Watson|              68.2498709344347|
|  GJ Maxwell|             67.90697674418605|
| BB McCullum|             67.77777777777777|
|   ML Hayden|             67.57000903342367|
|   HH Pandya|             66.86434395848777|
|  JC Buttler|             65.92765460910152|
|      N Rana|             65.41405706332637|
|  KA Pollard|             65.23321204101886|
|     RR Pant|             65.12746512746513|
| LMP Simmons|             64.87488415199259|
|Ishan Kishan|             64.73988439306359|
|   Q de Kock|             64.31852986217459|
|Yuvraj Singh|             64.0727

# Who are the top run getters by scoring runs in non-boundaries?

In [16]:
non_four_six=df.filter(~df.batsman_runs.isin(4,6))
non_four_and_six=non_four_six.groupBy('Batsman').agg(_sum('batsman_runs').alias("Runs")).sort('Runs',ascending=False)
non_four_and_six.show()

+--------------+----+
|       Batsman|Runs|
+--------------+----+
|       V Kohli|2650|
|      SK Raina|2232|
|      S Dhawan|2179|
|     RG Sharma|2114|
|      MS Dhoni|2084|
|     DA Warner|2044|
|     G Gambhir|1895|
|AB de Villiers|1879|
|    RV Uthappa|1813|
|     AM Rahane|1813|
|    KD Karthik|1685|
|     AT Rayudu|1635|
|     MK Pandey|1558|
|     SR Watson|1230|
|     YK Pathan|1202|
|     JH Kallis|1143|
|      CH Gayle|1142|
|     SPD Smith|1137|
|     SV Samson|1130|
|      PA Patel|1094|
+--------------+----+
only showing top 20 rows



# Which players have best strike rate in death overs?

In [17]:
death=df[ (df['overs'].between(15,19)) ]#filtering out wides and no balls, and taking  only powerplay overs(1-6) 
runs_1=death.groupBy('batsman').agg(_sum('batsman_runs').alias('Total_runs'))#finding out total runs by each batsman
balls_1=death.groupby('batsman').count()#finding out total balls faced by each batsman

In [18]:
death_over=runs_1.join(balls_1,on='batsman')#In pandas we could  just do strike_rate=runs*100/balls and we could get strike rate. We can't do same in pyspark,so first I will join the two result set and then find the strike rate
result_12=death_over.withColumn("Strike rate in death", col("Total_runs")*100/col("count")).sort('Strike rate in death',ascending=False)#Dividing the two columns and getting the strike rate the hard way.
death_runs=result_12[result_12['Total_runs'] > 200]#players who have more than 200 runs in deaths
death_runs.select('Batsman','Strike rate in death').show()

+--------------+--------------------+
|       Batsman|Strike rate in death|
+--------------+--------------------+
|AB de Villiers|  215.84415584415584|
|       RR Pant|  207.72058823529412|
|    AD Russell|  195.12761020881672|
|      CH Gayle|  193.05555555555554|
|    MEK Hussey|   189.1304347826087|
|        N Rana|   188.9763779527559|
|     DA Warner|  184.94983277591973|
|     SV Samson|  184.91379310344828|
|       V Kohli|  182.53968253968253|
|      CL White|  182.41206030150755|
|     SR Watson|   180.4780876494024|
|     DA Miller|  176.74418604651163|
|     RG Sharma|  176.16209773539927|
|       KK Nair|  175.48387096774192|
|     DJ Hussey|   175.2136752136752|
|      KL Rahul|  174.62686567164178|
|  F du Plessis|  173.29842931937173|
|     GJ Bailey|  172.10526315789474|
|   BCJ Cutting|  171.42857142857142|
|     JH Kallis|  170.56277056277057|
+--------------+--------------------+
only showing top 20 rows



# Which players have best strike rate overall?

In [19]:
runs_2=df.groupBy('batsman').agg(_sum('batsman_runs').alias('Total_runs'))
balls_2=df.groupby('batsman').count()#finding out total balls faced by each batsman
runs_balls=runs_2.join(balls_2,on='batsman')

In [20]:
strike=runs_balls.withColumn("Strike rate", col("Total_runs")*100/col("count")).sort('Strike rate',ascending=False)#Dividing the two columns and getting the strike rate .
strike_rate=strike[strike['Total_runs'] > 1000]
strike_rate.select('Batsman','Strike rate').show()

+--------------+------------------+
|       Batsman|       Strike rate|
+--------------+------------------+
|    AD Russell| 171.9954648526077|
|     HH Pandya| 150.3901895206243|
|      V Sehwag|148.82705946535734|
|    GJ Maxwell|148.56860809476802|
|AB de Villiers|148.56004901960785|
|       RR Pant|146.82203389830508|
|    JC Buttler|144.76351351351352|
|    KA Pollard|143.47413383958235|
|      CH Gayle|142.78874925194495|
|     DA Warner|137.57528148730034|
|     YK Pathan| 137.5107296137339|
|       CA Lynn|136.46055437100213|
|     DA Miller| 134.6433770014556|
|     SR Watson|134.14127423822714|
|  Ishan Kishan|133.51708930540244|
|  AC Gilchrist|133.05466237942122|
|      KL Rahul|133.01507537688443|
|      SK Raina|132.83840633506557|
|      MS Dhoni|132.60807328943602|
| KS Williamson|132.48772504091653|
+--------------+------------------+
only showing top 20 rows



# Which players have scored most runs in a single over?This only include runs scored by batsman ,doesn't include any extra's like wide or no ball.

In [21]:
df.groupby(['match_id','batsman','overs']).agg(_sum('batsman_runs').alias("Runs")).sort('Runs',ascending=False).show()

+--------+----------+-----+----+
|match_id|   batsman|overs|Runs|
+--------+----------+-----+----+
|  501247|  CH Gayle|    2|  36|
|  734047|  SK Raina|    5|  32|
| 1216527| R Tewatia|   17|  30|
|  335988|  V Sehwag|   12|  30|
|  980987|   V Kohli|   18|  30|
|  501260|  SE Marsh|   14|  30|
|  548327|  CH Gayle|   12|  30|
| 1082592| HH Pandya|   19|  28|
| 1175372|AD Russell|   18|  28|
|  598027|  CH Gayle|    4|  28|
| 1136609|JC Buttler|    2|  28|
| 1136586|   SS Iyer|   19|  28|
|  598027|  CH Gayle|    7|  28|
| 1216542|  N Pooran|    8|  28|
|  548318| JA Morkel|   18|  28|
| 1178402|JC Buttler|   12|  28|
| 1082612|GJ Maxwell|   14|  27|
| 1216547|KA Pollard|   16|  27|
|  598027|  CH Gayle|   14|  26|
| 1178410|    MM Ali|   15|  26|
+--------+----------+-----+----+
only showing top 20 rows



Overs here is over number in which batsman has scored the runs

In [22]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date

df1 = df1.withColumn('year',year(df1.date))#Here we have added a year colummn from date column

# Which Player has scored most runs in a particular year?

In [23]:
join_main=df.join(df1,on='match_id')
sum_bat=join_main.groupby(['Batsman','year']).sum('batsman_runs').sort("sum(batsman_runs)",ascending=False)
sum_bat.show()

+--------------+----+-----------------+
|       Batsman|year|sum(batsman_runs)|
+--------------+----+-----------------+
|       V Kohli|2016|              973|
|     DA Warner|2016|              848|
| KS Williamson|2018|              735|
|    MEK Hussey|2013|              733|
|      CH Gayle|2012|              733|
|      CH Gayle|2013|              708|
|     DA Warner|2019|              692|
|AB de Villiers|2016|              687|
|       RR Pant|2018|              684|
|      KL Rahul|2020|              670|
|    RV Uthappa|2014|              660|
|      KL Rahul|2018|              659|
|     DA Warner|2017|              641|
|       V Kohli|2013|              634|
|  SR Tendulkar|2010|              618|
|      S Dhawan|2020|              618|
|      SE Marsh|2008|              616|
|      CH Gayle|2011|              608|
|     AT Rayudu|2018|              602|
|      KL Rahul|2019|              593|
+--------------+----+-----------------+
only showing top 20 rows



# Who are the top scorer of ipl by each year

In [24]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import col


window_1 = Window.partitionBy('Year').orderBy(col('sum(batsman_runs)').desc())
res_41=sum_bat.withColumn('rank',rank().over(window_1))
top_scorer=res_41[res_41['rank']==1].sort('year')
top_scorer.show()

+-------------+----+-----------------+----+
|      Batsman|year|sum(batsman_runs)|rank|
+-------------+----+-----------------+----+
|     SE Marsh|2008|              616|   1|
|    ML Hayden|2009|              572|   1|
| SR Tendulkar|2010|              618|   1|
|     CH Gayle|2011|              608|   1|
|     CH Gayle|2012|              733|   1|
|   MEK Hussey|2013|              733|   1|
|   RV Uthappa|2014|              660|   1|
|    DA Warner|2015|              562|   1|
|      V Kohli|2016|              973|   1|
|    DA Warner|2017|              641|   1|
|KS Williamson|2018|              735|   1|
|    DA Warner|2019|              692|   1|
|     KL Rahul|2020|              670|   1|
+-------------+----+-----------------+----+



# Which players have finished as top scorer of ipl most number of times? 

In [25]:
top_scorer.groupby('Batsman').count().sort('count',ascending=False).show()

+-------------+-----+
|      Batsman|count|
+-------------+-----+
|    DA Warner|    3|
|     CH Gayle|    2|
|     SE Marsh|    1|
|   MEK Hussey|    1|
|     KL Rahul|    1|
|   RV Uthappa|    1|
|    ML Hayden|    1|
|      V Kohli|    1|
|KS Williamson|    1|
| SR Tendulkar|    1|
+-------------+-----+



# Which Player has taken most wickets in a particular year?


In [26]:
sum_wic=join_main.groupby(['bowler','year']).sum('is_wicket').sort("sum(is_wicket)",ascending=False)
sum_wic.show()

+---------------+----+--------------+
|         bowler|year|sum(is_wicket)|
+---------------+----+--------------+
|       DJ Bravo|2013|            34|
|    JP Faulkner|2013|            33|
|       K Rabada|2020|            30|
|       M Morkel|2012|            30|
|     SL Malinga|2011|            30|
|      SP Narine|2012|            29|
|         AJ Tye|2018|            28|
|       DJ Bravo|2015|            28|
|       K Rabada|2019|            28|
|        B Kumar|2017|            28|
|      JJ Bumrah|2020|            28|
|     JD Unadkat|2017|            27|
|  R Vinay Kumar|2013|            27|
|       TA Boult|2020|            26|
|       RP Singh|2009|            26|
|    Imran Tahir|2019|            26|
|      MM Sharma|2014|            26|
|      SP Narine|2013|            26|
|     SL Malinga|2015|            26|
|Harbhajan Singh|2013|            25|
+---------------+----+--------------+
only showing top 20 rows



# Who are the top wicket takers of ipl by each year?

In [27]:
window_2 = Window.partitionBy('Year').orderBy(col('sum(is_wicket)').desc())
res_42=sum_wic.withColumn('rank',rank().over(window_2))
top_wicket=res_42[res_42['rank']==1].sort('year')
top_wicket.show()

+-------------+----+--------------+----+
|       bowler|year|sum(is_wicket)|rank|
+-------------+----+--------------+----+
|Sohail Tanvir|2008|            24|   1|
|     RP Singh|2009|            26|   1|
|      PP Ojha|2010|            22|   1|
|   SL Malinga|2011|            30|   1|
|     M Morkel|2012|            30|   1|
|     DJ Bravo|2013|            34|   1|
|    MM Sharma|2014|            26|   1|
|     DJ Bravo|2015|            28|   1|
|      B Kumar|2016|            24|   1|
|      B Kumar|2017|            28|   1|
|       AJ Tye|2018|            28|   1|
|     K Rabada|2019|            28|   1|
|     K Rabada|2020|            30|   1|
+-------------+----+--------------+----+



# Which players have finished as top wicket of ipl most number of times?

In [28]:
top_wicket.groupby('bowler').count().sort('count',ascending=False).show()

+-------------+-----+
|       bowler|count|
+-------------+-----+
|     K Rabada|    2|
|      B Kumar|    2|
|     DJ Bravo|    2|
|       AJ Tye|    1|
|   SL Malinga|    1|
|Sohail Tanvir|    1|
|    MM Sharma|    1|
|     M Morkel|    1|
|      PP Ojha|    1|
|     RP Singh|    1|
+-------------+-----+



# Which players have scored most number of fifty?

In [29]:
fil=df.groupby(['match_id','batsman']).sum('batsman_runs')
fifty=fil[(fil['sum(batsman_runs)'] >= 50) & (fil['sum(batsman_runs)'] < 100)]
fifty.groupby('batsman').count().sort('count',ascending=False).show()

+--------------+-----+
|       batsman|count|
+--------------+-----+
|     DA Warner|   48|
|      S Dhawan|   41|
|     RG Sharma|   39|
|       V Kohli|   39|
|      SK Raina|   38|
|AB de Villiers|   38|
|     G Gambhir|   36|
|      CH Gayle|   31|
|     AM Rahane|   28|
|    RV Uthappa|   24|
|      MS Dhoni|   23|
|     SR Watson|   21|
|      KL Rahul|   21|
|      SE Marsh|   20|
|    KD Karthik|   19|
|     AT Rayudu|   19|
|     MK Pandey|   18|
|      DR Smith|   17|
|     JH Kallis|   17|
|  F du Plessis|   16|
+--------------+-----+
only showing top 20 rows



# Which players have scored most number of hundreds?

In [30]:
hun=fil[fil['sum(batsman_runs)'] >= 100]
hun.groupby('batsman').count().sort('count',ascending=False).show(10)

+--------------+-----+
|       batsman|count|
+--------------+-----+
|      CH Gayle|    6|
|       V Kohli|    5|
|     SR Watson|    4|
|     DA Warner|    4|
|AB de Villiers|    3|
|       M Vijay|    2|
|     AM Rahane|    2|
|   BB McCullum|    2|
|      S Dhawan|    2|
|     SV Samson|    2|
+--------------+-----+
only showing top 10 rows



# Which player has  highest score in a match?

In [31]:
fil.sort('sum(batsman_runs)',ascending=False).show()

+--------+--------------+-----------------+
|match_id|       batsman|sum(batsman_runs)|
+--------+--------------+-----------------+
|  598027|      CH Gayle|              175|
|  335982|   BB McCullum|              158|
|  829795|AB de Villiers|              133|
| 1216510|      KL Rahul|              132|
|  980987|AB de Villiers|              129|
|  548372|      CH Gayle|              128|
| 1136602|       RR Pant|              128|
|  419137|       M Vijay|              127|
| 1082627|     DA Warner|              126|
|  734047|      V Sehwag|              122|
|  501206|   PC Valthaty|              120|
|  501243|      V Sehwag|              119|
|  335990|     A Symonds|              117|
| 1136620|     SR Watson|              117|
|  829785|      CH Gayle|              117|
|  335983|    MEK Hussey|              116|
|  734049|       WP Saha|              115|
|  336019|      SE Marsh|              115|
|  336018| ST Jayasuriya|              114|
| 1175366|   JM Bairstow|       

# Which player has been dismissed for duck most number of times?

In [32]:
duck=fil[fil['sum(batsman_runs)'] == 0]
duck.groupby('batsman').count().sort('count',ascending=False).show()

+---------------+-----+
|        batsman|count|
+---------------+-----+
|      AM Rahane|   13|
|       PA Patel|   13|
|     KD Karthik|   12|
|Harbhajan Singh|   12|
|       R Ashwin|   12|
|      MK Pandey|   12|
|      RG Sharma|   12|
|  Mandeep Singh|   11|
|      G Gambhir|   11|
|      PP Chawla|   11|
|       S Dhawan|   11|
|     GJ Maxwell|   10|
|      AT Rayudu|   10|
|       R Sharma|   10|
|      YK Pathan|   10|
| AB de Villiers|    9|
|        NV Ojha|    9|
|        P Kumar|    9|
|       AJ Finch|    8|
|       DR Smith|    8|
+---------------+-----+
only showing top 20 rows



# Which player has scored most hundreds in single season?

In [33]:
most_year=join_main.groupby(['year','match_id','batsman']).sum('batsman_runs')
most_hun=most_year[most_year['sum(batsman_runs)'] >= 100]
most_hun.groupby(['batsman','year']).count().sort('count',ascending=False).show(10)

+----------------+----+-----+
|         batsman|year|count|
+----------------+----+-----+
|         V Kohli|2016|    4|
|        CH Gayle|2011|    2|
|         HM Amla|2017|    2|
|        S Dhawan|2020|    2|
|       SR Watson|2018|    2|
|       SR Watson|2013|    1|
|        CH Gayle|2012|    1|
|         M Vijay|2012|    1|
|      MA Agarwal|2020|    1|
|DPMD Jayawardene|2010|    1|
+----------------+----+-----+
only showing top 10 rows



# Which players have most ducks in a single season?

In [34]:
most_duck=join_main.groupby(['year','match_id','batsman']).sum('batsman_runs')
most_duck_year=most_duck[most_duck['sum(batsman_runs)']  == 0]
most_duck_year.groupby(['batsman','year']).count().sort('count',ascending=False).show(10)

+---------------+----+-----+
|        batsman|year|count|
+---------------+----+-----+
|       R Sharma|2013|    4|
|  Anureet Singh|2015|    4|
|       M Manhas|2011|    4|
|       HH Gibbs|2009|    4|
|       S Dhawan|2020|    4|
|      MK Pandey|2012|    4|
|C de Grandhomme|2017|    3|
|       R Sharma|2012|    3|
|       RV Gomez|2011|    3|
|        PP Shaw|2020|    3|
+---------------+----+-----+
only showing top 10 rows



# Which player has taken most wickets in single match?

In [35]:
df.groupby(['match_id','bowler']).sum('is_wicket').sort('sum(is_wicket)',ascending=False).show(10)

+--------+-------------+--------------+
|match_id|       bowler|sum(is_wicket)|
+--------+-------------+--------------+
|  598056|    DJG Sammy|             6|
| 1178394|    AS Joseph|             6|
|  980963|   AD Russell|             6|
|  336005|Sohail Tanvir|             6|
|  980979|      A Zampa|             6|
|  548311|    RA Jadeja|             5|
|  597998|    SP Narine|             5|
| 1136600|       AJ Tye|             5|
| 1082634|   JD Unadkat|             5|
|  501229|     I Sharma|             5|
+--------+-------------+--------------+
only showing top 10 rows



# Which player has conceded most number of runs  when bowled all 4 overs?

In [36]:
lst_runs=df.groupby(['match_id','bowler']).count()
extra=df.filter(~df.extras_type.isin('legbyes','byes'))
runs_con=extra.groupby(['match_id','bowler']).agg(_sum('total_runs').alias("Runs conceded in a single match"))
joinsss=lst_runs.join(runs_con,on=['bowler','match_id'])
joinsss[joinsss['count'] >= 24].sort('Runs conceded in a single match',ascending=False).show()

+----------------+--------+-----+-------------------------------+
|          bowler|match_id|count|Runs conceded in a single match|
+----------------+--------+-----+-------------------------------+
|    Basil Thampi| 1136611|   25|                             70|
|        I Sharma|  598051|   27|                             66|
|Mujeeb Ur Rahman| 1178423|   27|                             66|
|        UT Yadav|  598054|   25|                             65|
|  Sandeep Sharma|  734007|   28|                             65|
|          S Kaul| 1216538|   26|                             64|
|        VR Aaron|  548380|   26|                             63|
|        AB Dinda|  598011|   26|                             63|
|        MG Neser|  598064|   25|                             62|
|       SR Watson|  981019|   26|                             61|
|      TG Southee| 1175372|   25|                             61|
|      AS Rajpoot| 1216541|   28|                             60|
|       R 

# Which player has conceded least number of runs  when bowled all 4 overs?

In [37]:
joinsss[joinsss['count'] >= 24].sort('Runs conceded in a single match',ascending=True).show()

+-----------------+--------+-----+-------------------------------+
|           bowler|match_id|count|Runs conceded in a single match|
+-----------------+--------+-----+-------------------------------+
|          A Nehra|  392226|   24|                              6|
|        YS Chahal| 1175356|   24|                              6|
|       FH Edwards|  392184|   24|                              6|
|      Rashid Khan| 1216524|   24|                              7|
|      LH Ferguson| 1082624|   25|                              7|
|         R Sharma|  501241|   24|                              7|
|            B Lee|  501245|   24|                              8|
|        KV Sharma|  598030|   24|                              8|
|   M Muralitharan|  392234|   24|                              8|
|          P Kumar|  548350|   24|                              8|
|         DW Steyn|  548376|   24|                              8|
|   Mohammed Siraj| 1216494|   24|                            

# Which ballers have bowled most Deliveres in a single match?(Note :- A bowler bowls 24 Deliveres in match,anything over that are extra balls)

In [38]:
df.groupby(['match_id','bowler']).count().sort('count',ascending=False).show()

+--------+--------------+-----+
|match_id|        bowler|count|
+--------+--------------+-----+
|  392222|    SL Malinga|   31|
| 1216522|     R Tewatia|   30|
|  548377|       SW Tait|   30|
|  501270|      AN Ahmed|   30|
|  419107|       SW Tait|   30|
|  392210|    SM Harwood|   30|
|  336014|      Umar Gul|   30|
|  392222|         B Lee|   30|
|  392226|   MF Maharoof|   30|
| 1181766|      KMA Paul|   30|
| 1136603|     JC Archer|   30|
| 1082647|MJ McClenaghan|   29|
|  829709|    MG Johnson|   29|
| 1216537|      VR Aaron|   29|
| 1216525|     SM Curran|   29|
|  336040|     JA Morkel|   29|
|  392218|    SL Malinga|   29|
|  829737|     JJ Bumrah|   29|
|  548379| BW Hilfenhaus|   29|
|  392239|       P Kumar|   29|
+--------+--------------+-----+
only showing top 20 rows



Malinga has bowled 7 extra balls in a single match

# Which player has most number of dot balls in ipl?

In [39]:
dot=df[df['total_runs']==0]
dot.groupby(['bowler']).count().sort('count',ascending=False).show()

+---------------+-----+
|         bowler|count|
+---------------+-----+
|Harbhajan Singh| 1244|
|       R Ashwin| 1166|
|        B Kumar| 1155|
|     SL Malinga| 1144|
|      PP Chawla| 1137|
|       A Mishra| 1125|
|      SP Narine| 1100|
|        P Kumar| 1075|
|       DW Steyn| 1019|
|       UT Yadav|  954|
|         Z Khan|  873|
|      RA Jadeja|  870|
|       DJ Bravo|  860|
| Sandeep Sharma|  847|
|      IK Pathan|  835|
|       I Sharma|  817|
|      JJ Bumrah|  811|
|      SR Watson|  803|
|        A Nehra|  798|
|  R Vinay Kumar|  763|
+---------------+-----+
only showing top 20 rows



# Which player has most number of dot balls in a single ipl match?

In [40]:
dot.groupby(['match_id','bowler']).count().sort('count',ascending=False).show()

+--------+-------------+-----+
|match_id|       bowler|count|
+--------+-------------+-----+
| 1178398|    DL Chahar|   19|
|  392226|      A Nehra|   19|
|  392233|     MM Patel|   19|
|  548376|     DW Steyn|   18|
|  336023|Sohail Tanvir|   18|
|  501223|     M Morkel|   18|
|  981001|     DR Smith|   18|
| 1175356|    YS Chahal|   18|
| 1175356|  Imran Tahir|   18|
|  829801|       Z Khan|   18|
|  336029|     DW Steyn|   18|
|  733977|      B Kumar|   18|
|  598065|     A Mishra|   18|
|  392201|    DP Nannes|   18|
| 1136616|   AS Rajpoot|   18|
|  336018|   SM Pollock|   17|
|  548311|    RA Jadeja|   17|
|  548319|      P Kumar|   17|
|  598023|     RP Singh|   17|
|  336019|    IK Pathan|   17|
+--------+-------------+-----+
only showing top 20 rows



# Which player has most maidens in ipl?

In [41]:
maiden=df[(df['extras_type'].isin('byes','legbyes','NA')) &  (df['total_runs']==0) ]
maiden_over=maiden.groupby(['match_id','bowler','overs']).count()

In [42]:
res19=maiden_over[maiden_over['count'] >= 6]
res19.groupby('bowler').count().sort('count',ascending=False).show(1)

+-------+-----+
| bowler|count|
+-------+-----+
|P Kumar|   14|
+-------+-----+
only showing top 1 row



# Which players have bowled most number of  overs in ipl?

In [43]:
maxx=df.groupby(['match_id','bowler','overs']).count()
max_over=maxx[maxx['count'] >= 6]
resm=max_over.groupby('bowler').count().sort('count',ascending=False)
resm.show()

+---------------+-----+
|         bowler|count|
+---------------+-----+
|Harbhajan Singh|  561|
|      PP Chawla|  538|
|       R Ashwin|  535|
|       A Mishra|  525|
|     SL Malinga|  464|
|      SP Narine|  463|
|      RA Jadeja|  448|
|       DJ Bravo|  447|
|        B Kumar|  445|
|       UT Yadav|  419|
|        P Kumar|  417|
|         Z Khan|  364|
|       DW Steyn|  360|
|      YS Chahal|  353|
|  R Vinay Kumar|  350|
|      JJ Bumrah|  347|
| Sandeep Sharma|  341|
|      SR Watson|  336|
|       AR Patel|  335|
|      IK Pathan|  335|
+---------------+-----+
only showing top 20 rows



# Which bowler has best economic rate in ipl?(only those have bowled over 50 overs)

In [44]:
ext=df[~df['extras_type'].isin('byes','leg_byes')]#filetring out runs conceded by byes or leg byes
runs_concc=ext.groupby('bowler').sum('total_runs')

In [45]:
from pyspark.sql.functions import col

eco=resm.join(runs_concc,on='bowler')



bow_eco=eco.withColumn("economic rate", col('sum(total_runs)')/col("count")).sort('economic rate')
economic_rate=bow_eco[bow_eco['count'] > 50]
economic_rate.select('bowler','economic rate').show()

+-----------------+------------------+
|           bowler|     economic rate|
+-----------------+------------------+
|      Rashid Khan| 6.373983739837398|
|       GD McGrath| 6.777777777777778|
|         A Kumble| 6.849056603773585|
|        SP Narine|6.9049676025917925|
|   M Muralitharan| 6.909090909090909|
|Washington Sundar| 6.954128440366972|
|       DL Vettori|         6.9765625|
| RE van der Merwe|               7.0|
|         R Ashwin| 7.003738317757009|
|   AD Mascarenhas|7.0588235294117645|
|          J Botha| 7.060869565217391|
|         DW Steyn| 7.116666666666666|
|         R Sharma| 7.123376623376624|
|         CV Varun| 7.127272727272727|
|  Harbhajan Singh| 7.174688057040998|
|        R Tewatia| 7.189473684210526|
|        JC Archer| 7.294117647058823|
|         SK Warne| 7.341708542713568|
|         M Kartik| 7.366492146596858|
|        KH Pandya| 7.367149758454106|
+-----------------+------------------+
only showing top 20 rows



# Which bowlers  have best economic rate death overs?(only those have bowled over 30 overs)

In [46]:
death=df[df['overs'].between(15,19)]
maxx_death=death.groupby(['match_id','bowler','overs']).count()
max_over_death=maxx_death[maxx_death['count'] >= 6]
res_d=max_over_death.groupby('bowler').count()

In [47]:
de=df[(~df['extras_type'].isin('byes','leg_byes')) & (df['overs'].between(15,19))]#filetring out runs conceded by byes or leg byes and only death overs
runs_death=de.groupby('bowler').sum('total_runs')

eco_death=res_d.join(runs_death,on='bowler')



bow_eco_death=eco_death.withColumn("economic rate", col('sum(total_runs)')/col("count")).sort('economic rate')
economic_rate=bow_eco_death[bow_eco_death['count'] > 30]
economic_rate.select('bowler','economic rate').show()

+-----------------+------------------+
|           bowler|     economic rate|
+-----------------+------------------+
|     DE Bollinger|7.7368421052631575|
|        SP Narine| 7.827338129496403|
|         MA Starc|7.9393939393939394|
|         R Ashwin| 8.026666666666667|
|      Rashid Khan| 8.071428571428571|
|  Harbhajan Singh| 8.272727272727273|
|   M Muralitharan|               8.4|
|       SL Malinga| 8.458100558659218|
|Mustafizur Rahman| 8.555555555555555|
|        CH Morris| 8.641304347826088|
|         DW Steyn| 8.650485436893204|
|       WD Parnell|               9.0|
|        RA Jadeja|               9.0|
|        JJ Bumrah| 9.006849315068493|
|        KK Cooper| 9.022727272727273|
|         R Bhatia| 9.054054054054054|
|           S Kaul| 9.072727272727272|
|         SK Warne| 9.083333333333334|
|            B Lee| 9.098039215686274|
|          A Nehra| 9.126436781609195|
+-----------------+------------------+
only showing top 20 rows



# Which bowlers have best economic rate in powerplay ipl?(only those have bowled over 30 overs)

In [48]:
poww=df[df['overs'].between(0,5)]
poww_play=poww.groupby(['match_id','bowler','overs']).count()
poww_overs=poww_play[poww_play['count'] >= 6]
res_p=poww_overs.groupby('bowler').count()

In [49]:
po=df[(~df['extras_type'].isin('byes','leg_byes')) & (df['overs'].between(0,5))]#filetring out runs conceded by byes or leg byes and only death overs
po_runs=po.groupby('bowler').sum('total_runs')

eco_pow=res_p.join(po_runs,on='bowler')



bow_eco_pow=eco_pow.withColumn("economic rate", col('sum(total_runs)')/col("count")).sort('economic rate')
economic_rate_pow=bow_eco_pow[bow_eco_pow['count'] > 30]
economic_rate_pow.select('bowler','economic rate').show()


+-------------+------------------+
|       bowler|     economic rate|
+-------------+------------------+
|    JC Archer|5.4035087719298245|
|   GD McGrath| 5.891891891891892|
|   SM Pollock| 6.147058823529412|
|      B Kumar| 6.180672268907563|
|    SP Narine| 6.347107438016529|
|Iqbal Abdulla| 6.369565217391305|
|   WPUJC Vaas| 6.393939393939394|
|     DW Steyn| 6.417989417989418|
|   SL Malinga| 6.454054054054054|
|BW Hilfenhaus|               6.5|
|    DP Nannes| 6.543859649122807|
|     R Ashwin| 6.565789473684211|
|    JJ Bumrah| 6.672727272727273|
|    RJ Harris|  6.76056338028169|
|    YK Pathan| 6.796610169491525|
|      A Singh| 6.909090909090909|
|      P Kumar| 6.912213740458015|
|       Z Khan| 6.925581395348837|
|        B Lee|  7.08641975308642|
|    IC Pandey| 7.090909090909091|
+-------------+------------------+
only showing top 20 rows



# Which bowlers have taken most wickets in ipl?

In [50]:
highest_wicket_taker=df[~df['dismissal_kind'].isin (['obstructing the field','run out']) ]#filtering out obstructing the field and run out dismissal kind 
result_2=highest_wicket_taker.groupBy('bowler').sum('is_wicket').sort('sum(is_wicket)',ascending=False)
result_2.show()

+---------------+--------------+
|         bowler|sum(is_wicket)|
+---------------+--------------+
|     SL Malinga|           170|
|       A Mishra|           160|
|      PP Chawla|           156|
|       DJ Bravo|           153|
|Harbhajan Singh|           150|
|       R Ashwin|           138|
|        B Kumar|           136|
|      SP Narine|           127|
|      YS Chahal|           121|
|       UT Yadav|           119|
|      RA Jadeja|           114|
|      JJ Bumrah|           109|
| Sandeep Sharma|           109|
|        A Nehra|           106|
|  R Vinay Kumar|           105|
|         Z Khan|           103|
|       DW Steyn|            97|
|      SR Watson|            92|
|      MM Sharma|            92|
|       RP Singh|            90|
+---------------+--------------+
only showing top 20 rows



# Which bowlers have best bowling avg in ipl?(only those with 30+ wickets)

In [51]:
bowl_a=result_2.join(runs_concc,on='bowler')


bow_avg=bowl_a.withColumn("Bowling avg", col('sum(total_runs)')/col("sum(is_wicket)")).sort('Bowling avg')
bowling_avg=bow_avg[bow_avg['sum(is_wicket)'] > 30]
bowling_avg.select('bowler','Bowling avg').show()

+---------------+------------------+
|         bowler|       Bowling avg|
+---------------+------------------+
|       K Rabada|18.508196721311474|
|   DE Bollinger|18.657894736842106|
|     SL Malinga|20.341176470588234|
|    Rashid Khan|20.906666666666666|
|       MA Starc|21.205882352941178|
|    Imran Tahir|           21.3125|
|      JC Archer|21.565217391304348|
|         AJ Tye|            22.175|
|      YS Chahal|22.735537190082646|
|NM Coulter-Nile|22.975609756097562|
|       MM Patel| 23.31081081081081|
|      S Aravind|23.488888888888887|
|      KK Cooper|23.727272727272727|
|        A Nehra| 23.91509433962264|
|      RJ Harris|24.022222222222222|
|       A Kumble|              24.2|
|      JJ Bumrah|24.256880733944953|
|        S Gopal|            24.375|
|       A Mishra|          24.41875|
|        B Kumar| 24.46323529411765|
+---------------+------------------+
only showing top 20 rows



# Which bowlers have best bowling strike rate in ipl?

In [52]:
no_of_balls=df.groupby('bowler').count()
bow_str=result_2.join(no_of_balls,on='bowler')

In [53]:
bow_strike=bow_str.withColumn("Strike Rate", col('count')/col("sum(is_wicket)")).sort('Strike Rate')
bowling_strike=bow_strike[bow_strike['sum(is_wicket)'] > 30]
bowling_strike.select('bowler','Strike Rate').show()

+---------------+------------------+
|         bowler|       Strike Rate|
+---------------+------------------+
|       K Rabada|13.770491803278688|
|   DE Bollinger|15.789473684210526|
|         AJ Tye|            16.125|
|    Imran Tahir|            16.425|
|     SL Malinga| 17.49411764705882|
|      S Aravind|17.511111111111113|
|       MA Starc|              18.0|
|      YS Chahal|18.082644628099175|
|      KK Cooper|18.181818181818183|
|       TA Boult|18.285714285714285|
|NM Coulter-Nile|18.317073170731707|
|       DJ Bravo|18.601307189542485|
|        A Nehra| 18.62264150943396|
|      JC Archer|18.652173913043477|
|       MM Patel|18.675675675675677|
|        S Gopal|18.708333333333332|
| Mohammed Siraj|              18.9|
| MJ McClenaghan| 18.95774647887324|
|      CH Morris|             19.15|
|      RJ Harris| 19.22222222222222|
+---------------+------------------+
only showing top 20 rows



# Which bowler has most wickets in death overs in ipl?

In [54]:
death_wicket_taker=death[~death['dismissal_kind'].isin (['obstructing the field','run out']) ]#filtering out obstructing the field and run out dismissal kind 
death_wicket_taker.groupBy('bowler').sum('is_wicket').sort('sum(is_wicket)',ascending=False).show()


+--------------+--------------+
|        bowler|sum(is_wicket)|
+--------------+--------------+
|    SL Malinga|           108|
|      DJ Bravo|            95|
|       B Kumar|            79|
|     JJ Bumrah|            61|
|     SP Narine|            60|
|       A Nehra|            54|
|      DW Steyn|            50|
|      UT Yadav|            49|
|     CH Morris|            48|
| R Vinay Kumar|            47|
|      RP Singh|            45|
|    JD Unadkat|            44|
|     SR Watson|            42|
|     MM Sharma|            41|
|      K Rabada|            41|
|Sandeep Sharma|            38|
|      A Mishra|            38|
|     PP Chawla|            37|
|Mohammed Shami|            37|
|        Z Khan|            37|
+--------------+--------------+
only showing top 20 rows



# Which bowler has most wickets in power player overs in ipl?

In [55]:
power=df[df['overs'].between(0,5)]
power_wicket_taker=power[~power['dismissal_kind'].isin (['obstructing the field','run out']) ]#filtering out obstructing the field and run out dismissal kind 
power_wicket_taker.groupBy('bowler').sum('is_wicket').sort('sum(is_wicket)',ascending=False).show()

+--------------+--------------+
|        bowler|sum(is_wicket)|
+--------------+--------------+
|Sandeep Sharma|            53|
|        Z Khan|            53|
|       B Kumar|            48|
|      UT Yadav|            45|
|   DS Kulkarni|            44|
|      I Sharma|            43|
|      R Ashwin|            42|
|       P Kumar|            40|
|       A Nehra|            40|
|    SL Malinga|            37|
|      M Morkel|            36|
|      RP Singh|            35|
|     MM Sharma|            35|
|      DW Steyn|            34|
|     SR Watson|            34|
|     JA Morkel|            34|
|     DL Chahar|            33|
|      AB Dinda|            33|
|MJ McClenaghan|            31|
| R Vinay Kumar|            30|
+--------------+--------------+
only showing top 20 rows



# Which bowler has most 5 wickets haul in ipl?

In [56]:
fv=df[~df['dismissal_kind'].isin('obstructing the field','run out','retired hurt')]
five=fv.groupby(['match_id','bowler']).sum('is_wicket')
five_wickets=five[(five['sum(is_wicket)'] >= 5)]
five_wickets.groupby('bowler').count().sort('count',ascending=False).show(10)

+--------------+-----+
|        bowler|count|
+--------------+-----+
|    JD Unadkat|    2|
|   JP Faulkner|    2|
|    AS Rajpoot|    1|
|       B Kumar|    1|
|AD Mascarenhas|    1|
|      A Mishra|    1|
|      A Kumble|    1|
|    SL Malinga|    1|
|        AJ Tye|    1|
|      I Sharma|    1|
+--------------+-----+
only showing top 10 rows



# Which bowler has most wickets by bowling out the batsman?when batsman is bowled

In [57]:
bow_bowled=df[df['dismissal_kind']=='bowled']
bow_bowled.groupby('bowler').sum('is_wicket').sort('sum(is_wicket)',ascending=False).show()

+---------------+--------------+
|         bowler|sum(is_wicket)|
+---------------+--------------+
|     SL Malinga|            63|
|      PP Chawla|            43|
|      SP Narine|            36|
|        B Kumar|            36|
|      RA Jadeja|            29|
|Harbhajan Singh|            29|
|       DW Steyn|            26|
|      JJ Bumrah|            26|
|      YS Chahal|            25|
|       RP Singh|            25|
|       A Mishra|            25|
|       R Ashwin|            24|
|        A Nehra|            23|
|    Rashid Khan|            22|
|      IK Pathan|            22|
|       AR Patel|            22|
|         Z Khan|            22|
|       UT Yadav|            21|
|      MM Sharma|            21|
| Sandeep Sharma|            21|
+---------------+--------------+
only showing top 20 rows



# Which bowler has most wickets by lbw?

In [58]:
bow_lbw=df[df['dismissal_kind']=='lbw']
bow_lbw.groupby('bowler').sum('is_wicket').sort('sum(is_wicket)',ascending=False).show()

+---------------+--------------+
|         bowler|sum(is_wicket)|
+---------------+--------------+
|    Rashid Khan|            18|
|      PP Chawla|            17|
|      SP Narine|            17|
|       R Ashwin|            15|
| Sandeep Sharma|            14|
|       A Mishra|            13|
|      JJ Bumrah|            11|
|     SL Malinga|            11|
|      RA Jadeja|            11|
|         Z Khan|             9|
|       UT Yadav|             9|
| M Muralitharan|             9|
|       R Sharma|             8|
|Harbhajan Singh|             8|
|        B Kumar|             8|
|    Imran Tahir|             7|
|      JA Morkel|             7|
|       R Bhatia|             6|
|  Iqbal Abdulla|             6|
|      JH Kallis|             6|
+---------------+--------------+
only showing top 20 rows



# Which teams have bowled out the oppenets most number of times?(Taking 10 wickets in a match)

In [59]:
bowled=df.groupby(['match_id','bowling_team']).sum('is_wicket')
most_bowed=bowled[bowled['sum(is_wicket)'] == 10]
most_bowed.groupby('bowling_team').count().sort('count',ascending=False).show()

+--------------------+-----+
|        bowling_team|count|
+--------------------+-----+
|      Mumbai Indians|   26|
| Chennai Super Kings|   18|
|Royal Challengers...|   17|
|    Rajasthan Royals|   15|
| Sunrisers Hyderabad|   14|
|     Kings XI Punjab|   14|
|Kolkata Knight Ri...|   11|
|    Delhi Daredevils|   10|
|     Deccan Chargers|    6|
|       Gujarat Lions|    3|
|      Delhi Capitals|    2|
|Rising Pune Super...|    2|
|Kochi Tuskers Kerala|    1|
+--------------------+-----+



# Highest wicketakers by each Team

In [60]:
team_wic=df.groupby('bowling_team','bowler').sum('is_wicket').sort('sum(is_wicket)',ascending=False)
window_4 = Window.partitionBy('bowling_team').orderBy(col('sum(is_wicket)').desc())
team_high=team_wic.withColumn('rank',rank().over(window_4))
team_high[team_high['rank']==1].show()

+--------------------+-------------+--------------+----+
|        bowling_team|       bowler|sum(is_wicket)|rank|
+--------------------+-------------+--------------+----+
| Sunrisers Hyderabad|      B Kumar|           121|   1|
| Chennai Super Kings|     DJ Bravo|           126|   1|
|                  NA|   TG Southee|             3|   1|
|Rising Pune Super...|   JD Unadkat|            27|   1|
|     Deccan Chargers|      PP Ojha|            66|   1|
|Kochi Tuskers Kerala|R Vinay Kumar|            17|   1|
|    Rajasthan Royals|   SK Trivedi|            73|   1|
|       Gujarat Lions|  DS Kulkarni|            23|   1|
|Royal Challengers...|    YS Chahal|           124|   1|
|Kolkata Knight Ri...|    SP Narine|           143|   1|
|Rising Pune Super...|     AB Dinda|            13|   1|
|     Kings XI Punjab|    PP Chawla|            89|   1|
|       Pune Warriors|     R Sharma|            35|   1|
|    Delhi Daredevils|     A Mishra|            91|   1|
|      Delhi Capitals|     K Ra

# Highest run getters by each Team

In [61]:
team_runs=df.groupby('batting_team','batsman').sum('batsman_runs')
window_5 = Window.partitionBy('batting_team').orderBy(col('sum(batsman_runs)').desc())
team_high_runs=team_runs.withColumn('rank',rank().over(window_5))
team_high_runs[team_high_runs['rank']==1].show()

+--------------------+------------+-----------------+----+
|        batting_team|     batsman|sum(batsman_runs)|rank|
+--------------------+------------+-----------------+----+
| Sunrisers Hyderabad|   DA Warner|             3819|   1|
| Chennai Super Kings|    SK Raina|             4527|   1|
|Rising Pune Super...|   SPD Smith|              472|   1|
|     Deccan Chargers|AC Gilchrist|             1220|   1|
|Kochi Tuskers Kerala| BB McCullum|              357|   1|
|    Rajasthan Royals|   AM Rahane|             2810|   1|
|       Gujarat Lions|    SK Raina|              841|   1|
|Royal Challengers...|     V Kohli|             5878|   1|
|Kolkata Knight Ri...|   G Gambhir|             3035|   1|
|Rising Pune Super...|   AM Rahane|              480|   1|
|     Kings XI Punjab|    SE Marsh|             2477|   1|
|       Pune Warriors|  RV Uthappa|             1103|   1|
|    Delhi Daredevils|    V Sehwag|             2174|   1|
|      Delhi Capitals|    S Dhawan|             1139|   

# Which players have scored more than 1000 runs and has more than 20 wickets ?

In [62]:
all_rounder=total_runs.join(result_2,total_runs.Batsman==result_2.bowler,'inner')
all_rounder[(all_rounder['Runs'] > 1000) & (all_rounder['sum(is_wicket)'] > 20)].show()

+------------+----+------------+--------------+
|     Batsman|Runs|      bowler|sum(is_wicket)|
+------------+----+------------+--------------+
|  KA Pollard|3023|  KA Pollard|            60|
|    DR Smith|2385|    DR Smith|            26|
|   YK Pathan|3204|   YK Pathan|            42|
|   HH Pandya|1349|   HH Pandya|            42|
|   JH Kallis|2427|   JH Kallis|            65|
|  AD Russell|1517|  AD Russell|            61|
|    DJ Bravo|1490|    DJ Bravo|           153|
|    SK Raina|5368|    SK Raina|            25|
|Yuvraj Singh|2750|Yuvraj Singh|            36|
|   SR Watson|3874|   SR Watson|            92|
|   IK Pathan|1139|   IK Pathan|            80|
|   RA Jadeja|2159|   RA Jadeja|           114|
|   JP Duminy|2029|   JP Duminy|            23|
+------------+----+------------+--------------+



# Which teams have scored most runs in a single match?

In [63]:
high=df.groupby(['match_id','batting_team']).sum('total_runs').sort('sum(total_runs)',ascending=False)
high.show()

+--------+--------------------+---------------+
|match_id|        batting_team|sum(total_runs)|
+--------+--------------------+---------------+
|  598027|Royal Challengers...|            263|
|  980987|Royal Challengers...|            248|
|  419137| Chennai Super Kings|            246|
| 1136604|Kolkata Knight Ri...|            245|
|  335983| Chennai Super Kings|            240|
|  829795|Royal Challengers...|            235|
|  501260|     Kings XI Punjab|            232|
| 1178422|Kolkata Knight Ri...|            232|
| 1175366| Sunrisers Hyderabad|            231|
|  733987|     Kings XI Punjab|            231|
|  501223|    Delhi Daredevils|            231|
| 1082641|     Kings XI Punjab|            230|
| 1216515|      Delhi Capitals|            228|
|  980907|Royal Challengers...|            227|
| 1216527|    Rajasthan Royals|            226|
|  734047|     Kings XI Punjab|            226|
|  829785|Royal Challengers...|            226|
|  598051| Chennai Super Kings|         

# Which teams have scored 200+ plus runs  most number of times? 

In [64]:
plus_200=high[high['sum(total_runs)'] >= 200]
plus_200.groupby('batting_team').count().sort('count',ascending=False).show()

+--------------------+-----+
|        batting_team|count|
+--------------------+-----+
|Royal Challengers...|   19|
| Chennai Super Kings|   17|
|     Kings XI Punjab|   14|
|      Mumbai Indians|   14|
|Kolkata Knight Ri...|   12|
| Sunrisers Hyderabad|   12|
|    Rajasthan Royals|    9|
|    Delhi Daredevils|    5|
|      Delhi Capitals|    2|
|     Deccan Chargers|    1|
|       Gujarat Lions|    1|
+--------------------+-----+



# Which team has given most number of extra runs in ipl?

In [65]:
spark.sql(
"""
select bowling_team,SUM(extra_runs) AS extra_runs from balls
where extras_type  in ('wides','penalty','noballs')
group by bowling_team
order by extra_runs desc;

"""
).show()

+--------------------+----------+
|        bowling_team|extra_runs|
+--------------------+----------+
|      Mumbai Indians|    1085.0|
|Royal Challengers...|    1041.0|
|     Kings XI Punjab|     989.0|
|Kolkata Knight Ri...|     890.0|
|    Rajasthan Royals|     825.0|
| Chennai Super Kings|     793.0|
|    Delhi Daredevils|     750.0|
| Sunrisers Hyderabad|     452.0|
|     Deccan Chargers|     382.0|
|       Pune Warriors|     198.0|
|      Delhi Capitals|     148.0|
|       Gujarat Lions|     114.0|
|Rising Pune Super...|      81.0|
|Rising Pune Super...|      70.0|
|Kochi Tuskers Kerala|      64.0|
|                  NA|      10.0|
+--------------------+----------+



# Which player has given most number of extra runs in ipl?

In [66]:
spark.sql(
"""
select bowler,SUM(extra_runs) AS extra_runs from balls
where extras_type  in ('wides','penalty','noballs')
group by bowler
order by extra_runs desc;

"""
).show()

+---------------+----------+
|         bowler|extra_runs|
+---------------+----------+
|     SL Malinga|     177.0|
|        P Kumar|     145.0|
|       DJ Bravo|     139.0|
|       UT Yadav|     132.0|
|        B Kumar|     121.0|
|       I Sharma|     119.0|
|       R Ashwin|     119.0|
|       DW Steyn|     117.0|
|      SR Watson|     111.0|
|    DS Kulkarni|     108.0|
|      JA Morkel|     103.0|
|       RP Singh|     102.0|
|Harbhajan Singh|      99.0|
|     KA Pollard|      94.0|
|         Z Khan|      91.0|
|       AB Dinda|      86.0|
|       M Morkel|      84.0|
|        SW Tait|      83.0|
|    S Sreesanth|      82.0|
|      IK Pathan|      80.0|
+---------------+----------+
only showing top 20 rows



# Which player has conceded most number of sixes in ipl?

In [67]:
spark.sql(
"""
select bowler,count(*) from balls
where batsman_runs = '6'
group by bowler
order by count(1) desc;
;

"""
).show()

+---------------+--------+
|         bowler|count(1)|
+---------------+--------+
|      PP Chawla|     181|
|       A Mishra|     172|
|      RA Jadeja|     148|
|Harbhajan Singh|     142|
|       DJ Bravo|     138|
|       R Ashwin|     135|
|      YS Chahal|     135|
|       UT Yadav|     116|
|     JD Unadkat|     108|
|      SP Narine|     107|
|        P Kumar|     104|
|  R Vinay Kumar|      98|
|        PP Ojha|      97|
|       AR Patel|      96|
| Sandeep Sharma|      93|
|      KV Sharma|      91|
|        B Kumar|      88|
|     SL Malinga|      86|
|    Imran Tahir|      83|
|      IK Pathan|      82|
+---------------+--------+
only showing top 20 rows



# Which player has been run-out  most number of times?

In [68]:
spark.sql(
"""
select player_dismissed,count(*) from balls
where dismissal_kind == 'run out'
group by player_dismissed
order by count(1) desc
"""
).show()

+----------------+--------+
|player_dismissed|count(1)|
+----------------+--------+
|       G Gambhir|      16|
|        S Dhawan|      15|
|        SK Raina|      13|
|       AT Rayudu|      13|
|         M Vijay|      12|
|      KD Karthik|      12|
|  AB de Villiers|      12|
| Y Venugopal Rao|      11|
|       YK Pathan|      11|
|        PA Patel|      10|
|        A Mishra|      10|
|       RG Sharma|      10|
|         P Kumar|      10|
|        MS Dhoni|       9|
|    F du Plessis|       9|
|       IK Pathan|       9|
|      RV Uthappa|       9|
|       SR Watson|       9|
|        AM Nayar|       8|
|DPMD Jayawardene|       8|
+----------------+--------+
only showing top 20 rows



# Which batsman has been hit wicket most number of times?

In [69]:
spark.sql(
"""
select batsman,count(*) from balls
where dismissal_kind == 'hit wicket'
group by batsman
order by count(1) desc
"""
).show()

+-------------+--------+
|      batsman|count(1)|
+-------------+--------+
|    HH Pandya|       1|
|  SA Asnodkar|       1|
|     MA Khote|       1|
| Yuvraj Singh|       1|
|Misbah-ul-Haq|       1|
|   SP Jackson|       1|
|  Rashid Khan|       1|
|      R Parag|       1|
|    SS Tiwary|       1|
|     DJ Hooda|       1|
|    RA Jadeja|       1|
|    DA Warner|       1|
+-------------+--------+



# Which player has taken most number of caught and bowled?

In [70]:
spark.sql(
"""
select bowler,count(*) from balls
where dismissal_kind == 'caught and bowled'
group by bowler
order by count(1) desc
"""
).show()

+---------------+--------+
|         bowler|count(1)|
+---------------+--------+
|Harbhajan Singh|      11|
|       DJ Bravo|      11|
|      SP Narine|       7|
|     KA Pollard|       6|
|      PP Chawla|       6|
|     SL Malinga|       6|
|        S Gopal|       6|
|       SK Warne|       5|
|       S Nadeem|       5|
|     JD Unadkat|       5|
|    Imran Tahir|       5|
|      RA Jadeja|       5|
|  Kuldeep Yadav|       5|
|        B Kumar|       5|
|     A Chandila|       5|
|    DS Kulkarni|       4|
|       R Ashwin|       4|
|       AR Patel|       4|
|  R Vinay Kumar|       4|
|      YS Chahal|       3|
+---------------+--------+
only showing top 20 rows



# Which player has taken most number of catches?

In [71]:
spark.sql(
"""
select fielder,count(*) from balls
where dismissal_kind == 'caught'
group by fielder
order by count(1) desc
"""
).show()

+--------------+--------+
|       fielder|count(1)|
+--------------+--------+
|    KD Karthik|     118|
|      MS Dhoni|     113|
|AB de Villiers|     103|
|      SK Raina|      99|
|     RG Sharma|      88|
|    RV Uthappa|      87|
|    KA Pollard|      84|
|       V Kohli|      76|
|      S Dhawan|      73|
|     MK Pandey|      70|
|      PA Patel|      69|
|     DA Warner|      66|
|       NV Ojha|      65|
|      DJ Bravo|      64|
|     RA Jadeja|      63|
|       WP Saha|      62|
|     AM Rahane|      58|
|     AT Rayudu|      56|
|  F du Plessis|      54|
|     SV Samson|      52|
+--------------+--------+
only showing top 20 rows



# Which player has most stumpings?

In [72]:
spark.sql(
"""
select fielder,count(*) from balls
where dismissal_kind == 'stumped'
group by fielder
order by count(1) desc
"""
).show()

+--------------+--------+
|       fielder|count(1)|
+--------------+--------+
|      MS Dhoni|      39|
|    RV Uthappa|      32|
|    KD Karthik|      30|
|       WP Saha|      20|
|      PA Patel|      16|
|  AC Gilchrist|      16|
|     Q de Kock|      12|
|       RR Pant|      11|
|       NV Ojha|      10|
| KC Sangakkara|       9|
|AB de Villiers|       8|
|    SP Goswami|       7|
|     KM Jadhav|       7|
|      MS Bisla|       7|
|     SV Samson|       6|
|   BB McCullum|       6|
|       AP Tare|       5|
|     DH Yagnik|       5|
|      KL Rahul|       5|
|   YV Takawale|       4|
+--------------+--------+
only showing top 20 rows



# Which team has won most number of matches in IPL?

In [73]:
spark.sql(
"""
select winner,count(winner) from matches
group by winner
order by count(winner) desc 
""").show()

+--------------------+-------------+
|              winner|count(winner)|
+--------------------+-------------+
|      Mumbai Indians|          120|
| Chennai Super Kings|          106|
|Kolkata Knight Ri...|           99|
|Royal Challengers...|           91|
|     Kings XI Punjab|           88|
|    Rajasthan Royals|           81|
|    Delhi Daredevils|           67|
| Sunrisers Hyderabad|           66|
|     Deccan Chargers|           29|
|      Delhi Capitals|           19|
|       Gujarat Lions|           13|
|       Pune Warriors|           12|
|Rising Pune Super...|           10|
|Kochi Tuskers Kerala|            6|
|Rising Pune Super...|            5|
|                  NA|            4|
+--------------------+-------------+



# Which team has best winning percentage in IPL?

In [74]:
matches=spark.sql(
"""

select matches_played, count(*)
from ((select team1 as matches_played from matches) union all
      (select team2 from matches)
     ) matches
group by matches_played
""")

In [75]:
winnerss=spark.sql(
"""
select winner,count(winner) from matches
group by winner
""")

In [76]:
winning=matches.join(winnerss,winnerss.winner==matches.matches_played,'inner')
winning_per=winning.withColumn("Winning Percentage", col('count(winner)')*100/col("count(1)")).sort('Winning Percentage',ascending=False)
win=winning_per[winning_per['count(winner)'] >= 20]
win.select('winner','Winning Percentage').show()

+--------------------+------------------+
|              winner|Winning Percentage|
+--------------------+------------------+
| Chennai Super Kings|59.550561797752806|
|      Mumbai Indians| 59.11330049261084|
| Sunrisers Hyderabad|53.225806451612904|
|Kolkata Knight Ri...|           51.5625|
|    Rajasthan Royals| 50.31055900621118|
|Royal Challengers...|46.666666666666664|
|     Kings XI Punjab| 46.31578947368421|
|    Delhi Daredevils| 41.61490683229814|
|     Deccan Chargers|38.666666666666664|
+--------------------+------------------+



# Which team won the match by biggest margin?

In [77]:
spark.sql(
"""
select match_id,winner,result_margin from matches
where result_margin != 'NA'
order by result_margin desc 
""").show(5)

+--------+-------------------+-------------+
|match_id|             winner|result_margin|
+--------+-------------------+-------------+
|  419114|     Mumbai Indians|           98|
| 1216510|    Kings XI Punjab|           97|
| 1082599|   Delhi Daredevils|           97|
|  829753|Chennai Super Kings|           97|
|  729293|Chennai Super Kings|           93|
+--------+-------------------+-------------+
only showing top 5 rows



# Which player has most man of  match awards in netural venue?

In [78]:
spark.sql(
"""
select player_of_match,count(player_of_match) from matches
where neutral_venue = 1
group by player_of_match
order by count(player_of_match) desc 
""").show()

+----------------+----------------------+
| player_of_match|count(player_of_match)|
+----------------+----------------------+
|      GJ Maxwell|                     3|
|       YK Pathan|                     3|
|       JH Kallis|                     3|
|       JP Duminy|                     3|
|    Yuvraj Singh|                     2|
|       RG Sharma|                     2|
|  Sandeep Sharma|                     2|
|    SR Tendulkar|                     2|
|     LRPL Taylor|                     2|
|       MK Pandey|                     2|
|       ML Hayden|                     2|
|  M Muralitharan|                     2|
|   KC Sangakkara|                     2|
|        DR Smith|                     2|
|        SK Raina|                     2|
|  AB de Villiers|                     2|
|DPMD Jayawardene|                     2|
|    AC Gilchrist|                     2|
|        V Sehwag|                     1|
|        R Dravid|                     1|
+----------------+----------------

# Which team has won  the toss most number of times in IPL?

In [79]:
spark.sql(
"""
select toss_winner,count(toss_winner) from matches
group by toss_winner
order by count(toss_winner) desc 
""").show()

+--------------------+------------------+
|         toss_winner|count(toss_winner)|
+--------------------+------------------+
|      Mumbai Indians|               106|
|Kolkata Knight Ri...|                98|
| Chennai Super Kings|                97|
|Royal Challengers...|                87|
|    Rajasthan Royals|                87|
|     Kings XI Punjab|                85|
|    Delhi Daredevils|                80|
| Sunrisers Hyderabad|                57|
|     Deccan Chargers|                43|
|       Pune Warriors|                20|
|      Delhi Capitals|                20|
|       Gujarat Lions|                15|
|Kochi Tuskers Kerala|                 8|
|Rising Pune Super...|                 7|
|Rising Pune Super...|                 6|
+--------------------+------------------+



# Which player has won most player_of_match awards?

In [80]:
spark.sql(
"""
select player_of_match,count(player_of_match) from matches
group by player_of_match
order by count(player_of_match) desc 
""").show()

+---------------+----------------------+
|player_of_match|count(player_of_match)|
+---------------+----------------------+
| AB de Villiers|                    23|
|       CH Gayle|                    22|
|      RG Sharma|                    18|
|      DA Warner|                    17|
|       MS Dhoni|                    17|
|      YK Pathan|                    16|
|      SR Watson|                    16|
|       SK Raina|                    14|
|        V Kohli|                    13|
|      G Gambhir|                    13|
|     MEK Hussey|                    12|
|      AM Rahane|                    12|
|       DR Smith|                    11|
|       A Mishra|                    11|
|       V Sehwag|                    11|
|     KA Pollard|                    11|
|     AD Russell|                    11|
|      JH Kallis|                    10|
|      AT Rayudu|                    10|
|      SP Narine|                     9|
+---------------+----------------------+
only showing top

# Which team has given highest extra runs  in a match?

In [81]:
result_1=df.groupby(['match_id','bowling_team']).sum('extra_runs').sort('sum(extra_runs)',ascending=False)
result_1.show()

+--------+--------------------+---------------+
|match_id|        bowling_team|sum(extra_runs)|
+--------+--------------------+---------------+
|  335986|     Deccan Chargers|             28|
|  501260|Royal Challengers...|             27|
|  392227|      Mumbai Indians|             26|
|  829811|    Rajasthan Royals|             26|
|  336023|Royal Challengers...|             26|
|  733983|Royal Challengers...|             25|
|  392235|    Delhi Daredevils|             24|
|  336028|     Kings XI Punjab|             23|
|  501254|     Kings XI Punjab|             23|
| 1136591|Royal Challengers...|             23|
|  501251|      Mumbai Indians|             22|
|  419132|      Mumbai Indians|             21|
|  734037|Kolkata Knight Ri...|             21|
|  336005| Chennai Super Kings|             21|
|  829709|     Kings XI Punjab|             21|
|  336024|     Deccan Chargers|             20|
|  419153| Chennai Super Kings|             20|
| 1178423|     Kings XI Punjab|         

# Write a query to return a report for highest run scorer in matches which were affected by Duckworth-Lewis’s method (D/L method).

In [82]:
merged=df.join(df1,on='match_id')#merging df and df1 tables 

In [83]:
DL=merged[merged['method']=='D/L']#finding all matches which were effected by D/L method
sum1=DL.groupBy(['match_id','batsman']).sum('batsman_runs')#finding out total runs scored by every batsman in each match effected by D/L 



In [84]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import col


window = Window.partitionBy('match_id').orderBy(col('sum(batsman_runs)').desc())
res_4=sum1.withColumn('rank',rank().over(window))
res_4.filter(res_4.rank==1).show()

+--------+------------+-----------------+----+
|match_id|     batsman|sum(batsman_runs)|rank|
+--------+------------+-----------------+----+
|  501245| S Badrinath|               54|   1|
|  829807|MC Henriques|               57|   1|
| 1136578|     CA Lynn|               74|   1|
|  733993|  KD Karthik|               39|   1|
|  392183|    V Sehwag|               38|   1|
|  392183|      K Goel|               38|   1|
|  336025| Salman Butt|               73|   1|
| 1136566|   AM Rahane|               45|   1|
|  392186|    CH Gayle|               44|   1|
|  336022|    V Sehwag|               51|   1|
|  980999|     V Kohli|              113|   1|
|  980989|   YK Pathan|               37|   1|
| 1136592|     RR Pant|               69|   1|
|  829743|   DA Warner|               91|   1|
|  980997|   AM Rahane|               42|   1|
|  980943|    S Dhawan|               56|   1|
|  392214|   ML Hayden|               89|   1|
|  501215|    SK Raina|               50|   1|
| 1082648|   

# Write a query to return a report for highest strike rate by a batsman inpowerplay (1-6 overs)

In [85]:
legal=df[(~df['extras_type'].isin(['wides', 'noballs'])) & (df['overs'].between(0,5)) ]#filtering out wides and no balls, and taking  only powerplay overs(1-6) 
runs=legal.groupBy('batsman').sum('batsman_runs')#finding out total runs by each batsman
ball=legal.groupby('batsman').count()#finding out total balls faced by each batsman
balls=ball.withColumnRenamed("batsman","batsman_1")

In [86]:
joined=balls.join(runs, balls.batsman_1 == runs.batsman, 'inner')#In pandas we could  just do strike_rate=runs*100/balls and we could get strike rate. We can't do same in pyspark , so first I will join the two result set and then find the strike rate
result_4=joined.withColumn("Strike rate", col("sum(batsman_runs)")*100/col("count")).sort('Strike rate',ascending=False)#Dividing the two columns and getting the strike rate the hard way.
result_4.select('batsman','Strike rate').show(10)#Top 10 batsman with highest strike in powerplay

+--------------+------------------+
|       batsman|       Strike rate|
+--------------+------------------+
|     KK Cooper|             350.0|
|    AUK Pathan|             280.0|
|     CH Morris|242.85714285714286|
| Shahid Afridi|238.88888888888889|
|    AD Russell|             230.0|
|     K Gowtham|             200.0|
|AD Mascarenhas|             200.0|
|   Sunny Singh|             187.5|
|     SP Narine|176.56675749318802|
|     BJ Haddin|163.63636363636363|
+--------------+------------------+
only showing top 10 rows



# Write a query to get a list of top 10 players with the highest batting average 

In [87]:
player_dismissed=df.groupby('player_dismissed').count()#Counting number of times a player is dismissed
total_runs=df.groupBy('batsman').sum('batsman_runs')
joined_data=player_dismissed.join(total_runs, player_dismissed.player_dismissed == total_runs.batsman, 'inner') 


In [88]:
result_7=joined_data.withColumn("Batsmen Avg", col("sum(batsman_runs)")/col("count")).sort('Batsmen Avg',ascending=False)#Dividing runs by number of times they have been dismissed
result_7.select('batsman','Batsmen Avg').show(10)

+-------------+------------------+
|      batsman|       Batsmen Avg|
+-------------+------------------+
|   MN van Wyk|55.666666666666664|
|   RD Gaikwad|              51.0|
|     AC Voges|             45.25|
|     KL Rahul| 44.86440677966102|
|      HM Amla| 44.38461538461539|
|Iqbal Abdulla|              44.0|
|    DA Warner| 42.71544715447155|
|  JM Bairstow|41.578947368421055|
|     CH Gayle| 41.13793103448276|
|     MS Dhoni|40.991150442477874|
+-------------+------------------+
only showing top 10 rows



# Write a query to find out who has officiated (as an umpire) the most number of matches in IPL

In [89]:
spark.sql(
"""
select umpire, count(*)
from ((select umpire1 as umpire from matches) union all
      (select umpire2 from matches)
     ) matches
group by umpire
order by count(*) desc ;
""").show()
#Combing two similar columns by union and counting numbers on times they have officiated

+--------------------+--------+
|              umpire|count(1)|
+--------------------+--------+
|              S Ravi|     121|
|     HDPK Dharmasena|      94|
|        AK Chaudhary|      87|
|       C Shamshuddin|      82|
|           M Erasmus|      65|
|           CK Nandan|      57|
|         Nitin Menon|      57|
|          SJA Taufel|      55|
|           Asad Rauf|      51|
|         VA Kulkarni|      50|
|        BNJ Oxenford|      48|
|         CB Gaffaney|      47|
|           RJ Tucker|      46|
|         BR Doctrove|      42|
|         RE Koertzen|      41|
|           Aleem Dar|      38|
|           BF Bowden|      37|
|            NJ Llong|      37|
|KN Ananthapadmana...|      33|
|      RK Illingworth|      31|
+--------------------+--------+
only showing top 20 rows



# Find venue details of the match where V Kohli scored his highest individual runs in IPL

In [90]:
spark.sql(
"""
select m.match_id,first(v.venue),first(v.city),sum(b.batsman_runs) from balls as b
inner join matches as m on b.match_id=m.match_id
inner join venue as v on m.venue_id =v.venue_id
where b.batsman= 'V Kohli'
group by m.match_id
order by sum(b.batsman_runs) desc limit 1; 
""").show()
#First filtering in matches in which batsman is kohli,grouping by each match and finding total runs by each match and returning them in descending order and limit by 1


+--------+--------------------+-----------+---------------------------------+
|match_id|        first(venue)|first(city)|sum(CAST(batsman_runs AS DOUBLE))|
+--------+--------------------+-----------+---------------------------------+
|  980999|M.Chinnaswamy Sta...|  Bengaluru|                            113.0|
+--------+--------------------+-----------+---------------------------------+



# Which venue has hosted most number of matches?

In [91]:
spark.sql(
"""
select b.venue,count(b.venue_id) from matches a
inner join venue b on a.venue_id=b.venue_id
group by b.venue
order by 2 desc;
""").show(20)

+--------------------+---------------+
|               venue|count(venue_id)|
+--------------------+---------------+
|M.Chinnaswamy Sta...|             80|
|        Eden Gardens|             77|
|    Feroz Shah Kotla|             74|
|    Wankhede Stadium|             73|
|Rajiv Gandhi Inte...|             64|
|MA Chidambaram St...|             57|
|Sawai Mansingh St...|             47|
|Punjab Cricket As...|             35|
|Dubai Internation...|             33|
|Sheikh Zayed Stadium|             29|
|Maharashtra Crick...|             21|
|Punjab Cricket As...|             21|
|Sharjah Cricket S...|             18|
|Dr DY Patil Sport...|             17|
|Subrata Roy Sahar...|             17|
|           Kingsmead|             15|
|Dr. Y.S. Rajasekh...|             13|
|     SuperSport Park|             12|
|Sardar Patel Stad...|             12|
|   Brabourne Stadium|             11|
+--------------------+---------------+
only showing top 20 rows



# Highest scores in which stadium?

In [92]:
spark.sql(
"""
select c.venue,a.match_id,a.inning,SUM(a.total_runs) AS Runs from balls a 
inner join matches b on a.match_id=b.match_id
inner join venue c on b.venue_id=c.venue_id
group by c.venue,a.match_id,a.inning
order by Runs desc;
""").show()

+--------------------+--------+------+-----+
|               venue|match_id|inning| Runs|
+--------------------+--------+------+-----+
|M.Chinnaswamy Sta...|  598027|     1|263.0|
|M.Chinnaswamy Sta...|  980987|     1|248.0|
|MA Chidambaram St...|  419137|     1|246.0|
|Holkar Cricket St...| 1136604|     1|245.0|
|Punjab Cricket As...|  335983|     1|240.0|
|    Wankhede Stadium|  829795|     1|235.0|
|Himachal Pradesh ...|  501260|     1|232.0|
|        Eden Gardens| 1178422|     1|232.0|
|    Barabati Stadium|  733987|     1|231.0|
|Rajiv Gandhi Inte...| 1175366|     1|231.0|
|    Feroz Shah Kotla|  501223|     1|231.0|
|    Wankhede Stadium| 1082641|     1|230.0|
|Sharjah Cricket S...| 1216515|     1|228.0|
|M.Chinnaswamy Sta...|  980907|     1|227.0|
|    Wankhede Stadium|  734047|     1|226.0|
|Sharjah Cricket S...| 1216527|     2|226.0|
|M.Chinnaswamy Sta...|  829785|     1|226.0|
|MA Chidambaram St...|  419137|     2|223.0|
|    Wankhede Stadium| 1082641|     2|223.0|
|Rajiv Gan

# Which Venue saw most number of sixes?

In [93]:
spark.sql(
"""
select c.venue,count(*) AS Number_of_Sixes from balls a 
inner join matches b on a.match_id=b.match_id
inner join venue c on b.venue_id=c.venue_id
where a.batsman_runs = 6
group by c.venue
order by Number_of_Sixes desc;
""").show()

+--------------------+---------------+
|               venue|Number_of_Sixes|
+--------------------+---------------+
|M.Chinnaswamy Sta...|           1114|
|    Wankhede Stadium|            898|
|        Eden Gardens|            861|
|    Feroz Shah Kotla|            834|
|Rajiv Gandhi Inte...|            657|
|MA Chidambaram St...|            607|
|Sawai Mansingh St...|            363|
|Dubai Internation...|            358|
|Punjab Cricket As...|            307|
|Sharjah Cricket S...|            296|
|Maharashtra Crick...|            261|
|Sheikh Zayed Stadium|            254|
|Punjab Cricket As...|            219|
|Dr DY Patil Sport...|            173|
|Holkar Cricket St...|            155|
|Dr. Y.S. Rajasekh...|            154|
|Subrata Roy Sahar...|            141|
|Sardar Patel Stad...|            130|
|           Kingsmead|            130|
|     SuperSport Park|            120|
+--------------------+---------------+
only showing top 20 rows



# Which Venue saw most number of fours?

In [94]:
spark.sql(
"""
select c.venue,count(*) AS Number_of_Sixes from balls a 
inner join matches b on a.match_id=b.match_id
inner join venue c on b.venue_id=c.venue_id
where a.batsman_runs = 4
group by c.venue
order by Number_of_Sixes desc;
""").show()

+--------------------+---------------+
|               venue|Number_of_Sixes|
+--------------------+---------------+
|        Eden Gardens|           2175|
|M.Chinnaswamy Sta...|           2152|
|    Wankhede Stadium|           2095|
|    Feroz Shah Kotla|           2026|
|Rajiv Gandhi Inte...|           1661|
|MA Chidambaram St...|           1444|
|Sawai Mansingh St...|           1349|
|Punjab Cricket As...|           1040|
|Dubai Internation...|            851|
|Sheikh Zayed Stadium|            781|
|Punjab Cricket As...|            628|
|Maharashtra Crick...|            552|
|Sharjah Cricket S...|            415|
|Dr DY Patil Sport...|            390|
|Subrata Roy Sahar...|            382|
|   Brabourne Stadium|            361|
|           Kingsmead|            361|
|     SuperSport Park|            318|
|Saurashtra Cricke...|            305|
|Sardar Patel Stad...|            302|
+--------------------+---------------+
only showing top 20 rows



# Top run getters in each stadium

In [95]:
final_join=join_main.join(df2,on="venue_id")
v_high=final_join.groupby('venue','batsman').sum('batsman_runs')


window_5 = Window.partitionBy('venue').orderBy(col('sum(batsman_runs)').desc())
res_42=v_high.withColumn('rank',rank().over(window_5))
venue_scorer=res_42[res_42['rank']==1]
venue_scorer.show(22)

+--------------------+----------------+-----------------+----+
|               venue|         batsman|sum(batsman_runs)|rank|
+--------------------+----------------+-----------------+----+
|Dubai Internation...|        KL Rahul|              392|   1|
|Himachal Pradesh ...|        SE Marsh|              334|   1|
|Sardar Patel Stad...|       AM Rahane|              308|   1|
|Punjab Cricket As...|        SE Marsh|              806|   1|
|    Barabati Stadium|      RV Uthappa|              152|   1|
|Punjab Cricket As...|        KL Rahul|              448|   1|
|       Nehru Stadium|DPMD Jayawardene|              111|   1|
|Maharashtra Crick...|       SPD Smith|              452|   1|
|        Eden Gardens|       G Gambhir|             1407|   1|
|     OUTsurance Oval|  AB de Villiers|               90|   1|
|    Feroz Shah Kotla|        V Sehwag|              933|   1|
|Rajiv Gandhi Inte...|       DA Warner|             1602|   1|
|   Brabourne Stadium|    SR Tendulkar|              26

# Most wickets in each stadium?

In [96]:
v_wick=final_join.groupby('venue','bowler').sum('is_wicket')


window_6 = Window.partitionBy('venue').orderBy(col('sum(is_wicket)').desc())
res_43=v_wick.withColumn('rank',rank().over(window_6))
venue_wicket=res_43[res_43['rank']==1].sort('sum(is_wicket)',ascending=False)
venue_wicket.show(40)

+--------------------+--------------+--------------+----+
|               venue|        bowler|sum(is_wicket)|rank|
+--------------------+--------------+--------------+----+
|    Wankhede Stadium|    SL Malinga|            73|   1|
|        Eden Gardens|     SP Narine|            61|   1|
|    Feroz Shah Kotla|      A Mishra|            60|   1|
|M.Chinnaswamy Sta...|     YS Chahal|            52|   1|
|MA Chidambaram St...|      R Ashwin|            50|   1|
|MA Chidambaram St...|      DJ Bravo|            50|   1|
|Sawai Mansingh St...|    SK Trivedi|            41|   1|
|Rajiv Gandhi Inte...|       B Kumar|            40|   1|
|Punjab Cricket As...|     PP Chawla|            24|   1|
|Punjab Cricket As...|Sandeep Sharma|            17|   1|
|Maharashtra Crick...|   Imran Tahir|            16|   1|
|Dubai Internation...|     YS Chahal|            13|   1|
|Subrata Roy Sahar...|      R Sharma|            13|   1|
|Himachal Pradesh ...|     PP Chawla|            13|   1|
|Dubai Interna